**写在前面**：本节内容是 [Datawhale三月的组队学习 - 集成学习（上）- CH2-机器学习基础模型回顾 -【Task4 掌握回归模型的评估及超参数调优】](https://github.com/datawhalechina/team-learning-data-mining/blob/master/EnsembleLearning/CH2-%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80%E6%A8%A1%E5%9E%8B%E5%9B%9E%E9%A1%BE/%E7%AC%AC%E4%BA%8C%E7%AB%A0%EF%BC%9A%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80.ipynb) 的学习笔记，对应notebook的  节，学习周期天（后调整为2天）

## 导入库和数据

In [87]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR # 由前面的测试可以发现，默认参数的SVR的回归效果最差，这里通过学习对其进行超参数调参
from sklearn import datasets
from sklearn.model_selection  import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler # 标准化
from sklearn.pipeline import Pipeline
# Day1
from sklearn.model_selection import GridSearchCV # 网格搜索
from sklearn.model_selection import RandomizedSearchCV # 随机搜索
from scipy.stats import uniform  # 引入均匀分布设置参数
# Day2
from bayes_opt import BayesianOptimization # 贝叶斯优化
from sklearn.model_selection import cross_val_score
import sko # 集成了几个启发式算法的包

In [20]:
# boston数据集作为本笔记的实验数据
boston = datasets.load_boston()
X = boston.data
y = boston.target
features = boston.feature_names
boston_data = pd.DataFrame(X,columns=features)
boston_data["Price"] = y

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25) # 数据切分

In [32]:
# 使用未调参的SVR作为调参实验模型
pipe_svr = Pipeline([("StandardScaler",StandardScaler()), ("svr",SVR())])
pipe_svr.fit(X_train, y_train)
y_pred = pipe_svr.predict(X_test)
print('MSE:',MSE(y_pred, y_test))

MSE: 30.161150722632946


## Day1

### 基础概念
**参考资料**：
- [《机器学习模型的超参数优化》- DeepHub公众号翻译](https://baijiahao.baidu.com/s?id=1665683313439416501&wfr=spider&for=pc)
- [4种主流超参数调优技术](https://zhuanlan.zhihu.com/p/234509605)
- [贝叶斯优化 Bayesian Optimization](https://blog.csdn.net/qq_40597317/article/details/80888837)
- [模型参数优化（一）：遗传算法](https://blog.csdn.net/qq_27586341/article/details/93622507)
- [模型参数优化（二）：粒子群优化](https://blog.csdn.net/qq_27586341/article/details/93634155)
- [模型参数优化（三）：模拟退火](https://blog.csdn.net/qq_27586341/article/details/93634978)

#### 超参数
超参数是在开始学习过程之前设置值的参数，其值无法从数据中估计。相反，其他参数的值通过训练得出。
  - 超参数通常用于帮助估计模型参数。
  - 超参数通常由人工指定。
  - 超参数通常可以使用启发式设置。
  - 超参数经常被调整为给定的预测建模问题。  

#### 调优方法简要介绍
1. **网格搜索 GridSearch**  
    搜索的思想非常简单，假设有2个超参数需要去选择，就把所有的超参数选择列出来分别做排列组合。假设有:
    $$\lambda = {0.01, 0.1, 1.0}$$
    $$\alpha = {0.01, 0.1, 1.0}$$
    可以以此做一个排列组合，然后针对每组超参数分别建立一个模型，然后选择测试误差最小的那组超参数。  
    换句话说，我们需要从超参数空间中寻找最优的超参数，很像一个网格中找到一个最优的节点，因此叫`网格搜索`。
    - 【优点】：可以找到期望中的最优组合。
    - 【缺点】：需要搜索的空间太大，费时费资源。尤其是组合情况呈指数级增长，使得很多情况下无法使用。
2. **随机搜索 RandomizedSearch**
    随机搜索中的每个参数都是从可能的参数值的分布中进行采样。
    - 【优点】：可以独立于参数数量和可能的值来选择计算成本;添加不影响性能的参数不会降低效率。
    - 【缺点】：可能找不到最优的组合
3. **基于梯度的优化方法**  
    通常用于神经网络内，应用场景并不广泛。主要限制有：
    - 超参数优化通常不是一个平滑的过程
    - 超参数优化往往具有非凸的性质
4. **贝叶斯优化**
    - 【优点】：既不需要求导，也不要求凸函数；只需要不断取样，来推测函数的最大值，且采样的点也不多
    - 【缺点】：容易陷入局部最优；资源消耗大；高维灾难
5. **遗传算法**
6. **粒子群优化**
7. **模拟退火**

### 调优方法
**参考资料**：
- [sklearn-model_evaluation-scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

#### sklearn里的网格搜索
**sklearn参数**:
- `param_grid`:传入一个字典，形如：{'参数名':[参数序列]}。
- `scoring`:用于评分的策略。支持输入：
    - 单个策略：
        - 字符串的形式，具体名称可参阅[评分参数](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
        - a callable that returns a single value.(可调用对象，应该是函数这些)
    - 多个策略：
        - 唯一字符串的列表或元组
        - 可调用的返回字典，其中的键是度量标准名称，值是度量标准分数
        - 以度量名称作为键并可调用a值的字典。
- `cv`:K折交叉检验的折数  
**注**:参数未全部列出，详细可见参考文档

In [65]:
pipe_svr = Pipeline([("StandardScaler",StandardScaler()), ("svr",SVR())])
param_range = [0.01, 0.1, 1.0, 10.0]
param_grid = {"svr__C":param_range, 
              "svr__gamma":param_range, 
              "svr__kernel":["linear","rbf"]}
gs = GridSearchCV(pipe_svr, 
                  param_grid, 
                  scoring = 'neg_mean_squared_error', 
                  cv = 10)
gs.fit(X_train, y_train)
print("最优得分: ", -gs.best_score_) # 这里因为要寻找最大化目标，所以对均方误差进行取反
print("最优组合: ", gs.best_params_)
print("组合个数: ", len(gs.cv_results_['params']))

最优得分:  16.18467140142489
最优组合:  {'svr__C': 10.0, 'svr__gamma': 0.1, 'svr__kernel': 'rbf'}
组合个数:  32


#### sklearn里的随机搜索
**sklearn参数**: 和网格搜索相一致，这里不再说明。

In [64]:
pipe_svr = Pipeline([("StandardScaler",StandardScaler()), ("svr",SVR())])
param_range = uniform(loc=0, scale=10)
param_grid = {"svr__C":param_range, 
              "svr__gamma":param_range, 
              "svr__kernel":["linear","rbf"]}
rs = RandomizedSearchCV(pipe_svr, 
                        param_grid, 
                        scoring = 'neg_mean_squared_error', 
                        cv = 10,
                        n_iter=32 # 搜索次数与网格搜索保持一致)
rs.fit(X_train, y_train)
print("最优得分: ", -rs.best_score_) # 这里因为要寻找最大化目标，所以对均方误差进行取反
print("最优组合: ", rs.best_params_)
print("组合个数: ", len(rs.cv_results_['params']))

最优得分:  26.28265445223402
最优组合:  {'svr__C': 9.004510129566087, 'svr__gamma': 9.472441206770952, 'svr__kernel': 'linear'}
组合个数:  32


## Day2

### 课程外的优化算法
**参考资料**:
- [调参神器贝叶斯优化（bayesian-optimization）实战篇](https://www.jianshu.com/p/92d8943fb0ba)

#### 贝叶斯优化

In [85]:
def svr_cv(C, gamma):
    res =cross_val_score(
        SVR(C = float(C),
            gamma = float(gamma),
            kernel = 'rbf'),
        X, y, scoring='neg_mean_squared_error', cv=10
    ).mean()
    return res

svr_op = BayesianOptimization(
        svr_cv,
        {'C': (0.1, 10.0),
        'gamma':  (0.1, 10.0)}
    )

svr_op.maximize()
print(svr_op.max)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        | -92.43    |  4.868    |  5.639    |
|  2        | -92.69    |  5.433    |  9.723    |
|  3        | -92.82    |  6.015    |  5.96     |
|  4        | -91.96    |  4.113    |  4.807    |
|  5        | -91.56    |  2.9      |  8.001    |
|  6        | -91.43    |  2.262    |  6.724    |
|  7        | -92.46    |  0.1      |  8.552    |
|  8        | -91.63    |  1.508    |  4.585    |
|  9        | -91.43    |  2.37     |  1.455    |
|  10       | -92.5     |  0.1      |  0.1      |
|  11       | -92.12    |  4.398    |  1.115    |
|  12       | -91.45    |  2.473    |  2.934    |
|  13       | -89.89    |  10.0     |  0.1      |
|  14       | -92.64    |  10.0     |  1.642    |
|  15       | -89.93    |  9.649    |  0.1      |
|  16       | -90.1     |  8.531    |  0.1      |
|  17       | -90.55    |  7.068    |  0.1      |
|  18       | -92.64    |  10.0     |  10.0     |


#### 其他方法
其他的启发式算法好像没发在这个问题上使用，没有找到合适的例子。。